In [1]:
!pip install pydantic pydantic-ai python-dotenv ipython devtools

In [2]:
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variables
openai_api_key = os.environ.get("OPENAI_API_KEY", "")
google_api_key = os.environ.get("GEMINI_API_KEY", "")

# Set API keys only if they're not already set
if not os.environ.get("OPENAI_API_KEY") and openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if not os.environ.get("GEMINI_API_KEY") and google_api_key:
    os.environ["GEMINI_API_KEY"] = google_api_key

# Add error handling for missing API keys
# Check which API keys are missing and provide specific warnings
missing_keys = []
if not os.environ.get("OPENAI_API_KEY"):
    missing_keys.append("OPENAI_API_KEY")
if not os.environ.get("GEMINI_API_KEY"):
    missing_keys.append("GEMINI_API_KEY")

if missing_keys:
    print(f"Warning: The following API key(s) are missing: {', '.join(missing_keys)}")
    print(f"Some functionality using {', '.join(missing_keys)} may be limited.")
# Display a message indicating that the API keys have been loaded
display(Markdown("### API keys have been loaded successfully."))

USE_GEMINI= True



Some functionality using OPENAI_API_KEY may be limited.


### API keys have been loaded successfully.

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
from pydantic import BaseModel, Field
from typing import Optional
from devtools import debug
from dataclasses import dataclass


class YearlyValue(BaseModel):
    """Model for values that can have current and previous year figures"""
    CFY: float
    PFY: Optional[float] = None


class StatementOfProfitOrLoss(BaseModel):
    """Financial statement model for profit or loss statement with current and previous year values"""
    
    # Fields in specified order
    revenue: YearlyValue = Field(..., description="Total revenue generated")
    other_income: Optional[YearlyValue] = Field(None, description="Income from sources other than main operations")
    employee_benefits_expense: Optional[YearlyValue] = Field(None, description="Expenses related to employee benefits")
    depreciation_expense: Optional[YearlyValue] = Field(None, description="Depreciation expenses")
    amortisation_expense: Optional[YearlyValue] = Field(None, description="Amortisation expenses")
    repairs_maintenance_expense: Optional[YearlyValue] = Field(None, description="Repairs and maintenance expenses")
    sales_marketing_expense: Optional[YearlyValue] = Field(None, description="Sales and marketing expenses")
    other_expenses: Optional[YearlyValue] = Field(None, description="Other operational expenses")
    other_gains_losses: Optional[YearlyValue] = Field(None, description="Other gains or losses")
    finance_costs_net: Optional[YearlyValue] = Field(None, description="Net finance costs")
    share_of_profit_loss_associates: Optional[YearlyValue] = Field(None, description="Share of profit or loss of associates and joint ventures")
    profit_loss_before_taxation: YearlyValue = Field(..., description="Profit or loss before taxation from continuing operations")
    income_tax_expense_benefit: YearlyValue = Field(..., description="Income tax expense or benefit from continuing operations")
    profit_loss_discontinued_operations: Optional[YearlyValue] = Field(None, description="Profit or loss from discontinued operations, net of taxation")
    total_profit_loss: Optional[YearlyValue] = Field(None, description="Total profit or loss, net of taxation for the year")
    profit_loss_attributable_to_owners: Optional[YearlyValue] = Field(None, description="Profit or loss attributable to owners of company")
    profit_loss_attributable_to_non_controlling: Optional[YearlyValue] = Field(None, description="Profit or loss attributable to non-controlling interests")


In [ ]:
# Define the system prompt with clearer field mapping instructions
FINANCIAL_STATEMENT_PROMPT = """You are a Singapore financial reporting specialist who converts annual reports to XBRL format.
Your task is to extract and map financial data from input reports into standardized Statement of Profit or Loss fields.

IMPORTANT MAPPING GUIDELINES:
1. For each financial item, extract both Current Financial Year (CFY) and Previous Financial Year (PFY) values when available.
   - CFY values are required for mandatory fields
   - PFY values should be included when available, but can be omitted if not present

2. IMPORTANT: Map ALL available fields in the input data to their corresponding fields in the output model.
   Do not ignore any fields that have a matching output field.

3. Pay close attention to financial terminology and map fields based on accounting meaning rather than exact names.
   Here are the specific field mappings you should look for:
   - "Revenue" → revenue
   - "Other Income" → other_income
   - "Employee Benefits Expense" → employee_benefits_expense
   - "Depreciation of Property, Plant and Equipment" → depreciation_expense
   - "Amortisation of Intangible Assets" → amortisation_expense
   - "Repairs and Maintenance" → repairs_maintenance_expense
   - "Sales and Marketing" → sales_marketing_expense
   - "Other Operating Expenses" → other_expenses
   - "Foreign Exchange Gain/(Loss)" → other_gains_losses
   - "Finance Costs" → finance_costs_net
   - "Share of Results of Associates" → share_of_profit_loss_associates
   - "Profit Before Tax" → profit_loss_before_taxation
   - "Income Tax Expense" → income_tax_expense_benefit
   - "Loss From Discontinued Operations" → profit_loss_discontinued_operations
   - "Profit For The Year" → total_profit_loss
   - "Owners of the Company" (under "Attributable to") → profit_loss_attributable_to_owners
   - "Non-controlling Interests" (under "Attributable to") → profit_loss_attributable_to_non_controlling

4. The three required fields that must have at least CFY values are:
   - revenue
   - profit_loss_before_taxation
   - income_tax_expense_benefit
   
5. Include ALL other fields in the output when they're present in the input data.
"""

In [19]:
dummy_data = {
    "revenue": 1000000.0,
    "other_income": 50000.0,
    "staff_costs": 350000.0,  # should map to employee_benefits_expense
    "depreciation": 75000.0,  # should map to depreciation_expense
    "amortization": 25000.0,  # should map to amortisation_expense
    "repairs_and_maintenance": 30000.0,  # should map to repairs_maintenance_expense
    "marketing_expenses": 120000.0,  # should map to sales_marketing_expense
    "general_expenses": 80000.0,  # should map to other_expenses
    "foreign_exchange_gain": 15000.0,  # should map to other_gains_losses
    "interest_expense": 40000.0,  # should map to finance_costs_net
    "share_of_results_from_associates": 35000.0,  # should map to share_of_profit_loss_associates
    "profit_before_tax": 380000.0,  # should map to profit_loss_before_taxation
    "tax_expense": 65000.0,  # should map to income_tax_expense_benefit
    "loss_from_discontinued_ops": 10000.0,  # should map to profit_loss_discontinued_operations
    "net_profit": 305000.0,  # should map to total_profit_loss
    "profit_attributable_to_shareholders": 290000.0,  # should map to profit_loss_attributable_to_owners
    "profit_attributable_to_minority_interests": 15000.0,  # should map to profit_loss_attributable_to_non_controlling
    
    # Some noise fields
    "company_name": "Test Company Ltd",
    "year": 2024,
    "current_ratio": 1.5
}

In [20]:
dummy_profit_loss_data = {
    "Revenue": {
        "CFY": 12500000,
        "PFY": 11250000
    },
    "Other Income": {
        "CFY": 350000,
        "PFY": 275000
    },
    "Employee Benefits Expense": {
        "CFY": -4750000,
        "PFY": -4250000
    },
    "Depreciation of Property, Plant and Equipment": {
        "CFY": -780000,
        "PFY": -720000
    },
    "Amortisation of Intangible Assets": {
        "CFY": -180000,
        "PFY": -165000
    },
    "Repairs and Maintenance": {
        "CFY": -320000,
        "PFY": -280000
    },
    "Sales and Marketing": {
        "CFY": -1450000,
        "PFY": -1350000
    },
    "Other Operating Expenses": {
        "CFY": -970000,
        "PFY": -850000
    },
    "Foreign Exchange Gain/(Loss)": {
        "CFY": 85000,
        "PFY": -120000
    },
    "Finance Costs": {
        "CFY": -325000,
        "PFY": -290000
    },
    "Share of Results of Associates": {
        "CFY": 620000,
        "PFY": 580000
    },
    "Profit Before Tax": {
        "CFY": 4780000,
        "PFY": 4080000
    },
    "Income Tax Expense": {
        "CFY": -860000,
        "PFY": -735000
    },
    "Profit From Continuing Operations": {
        "CFY": 3920000,
        "PFY": 3345000
    },
    "Loss From Discontinued Operations": {
        "CFY": -150000,
        "PFY": -275000
    },
    "Profit For The Year": {
        "CFY": 3770000,
        "PFY": 3070000
    },
    "Attributable to:": {
        "Owners of the Company": {
            "CFY": 3620000,
            "PFY": 2950000
        },
        "Non-controlling Interests": {
            "CFY": 150000,
            "PFY": 120000
        }
    },
    "Earnings Per Share (in cents)": {
        "Basic": {
            "CFY": 18.5,
            "PFY": 15.1
        },
        "Diluted": {
            "CFY": 18.2,
            "PFY": 14.8
        }
    },
    "Company Information": {
        "Name": "Singapore Ventures Ltd",
        "Financial Year": "2024",
        "Reporting Currency": "SGD"
    }
}

In [21]:
from pydantic_ai import Agent, ModelRetry
# import logfire
# logfire.configure()

model_name = 'google-gla:gemini-1.5-pro' if USE_GEMINI else 'openai:gpt-4o-mini'

# Define the agent
financial_statement_agent = Agent(
    model_name,
    result_type=StatementOfProfitOrLoss,
    system_prompt=FINANCIAL_STATEMENT_PROMPT
)

In [22]:
import json
# Convert the dummy data to a JSON string (optionally, you can use indent=4 for readability)
dummy_data_json = json.dumps(dummy_profit_loss_data, indent=4)

# Use an f-string to inject the dummy data into your prompt:
result = await financial_statement_agent.run(
    f'please map my data here: {dummy_data_json}'
)

In [23]:
print(result.data)

revenue=YearlyValue(CFY=12500000.0, PFY=11250000.0) other_income=YearlyValue(CFY=350000.0, PFY=275000.0) employee_benefits_expense=None depreciation_expense=None amortisation_expense=None repairs_maintenance_expense=None sales_marketing_expense=None other_expenses=None other_gains_losses=None finance_costs_net=None share_of_profit_loss_associates=None profit_loss_before_taxation=YearlyValue(CFY=4780000.0, PFY=4080000.0) income_tax_expense_benefit=YearlyValue(CFY=-860000.0, PFY=-735000.0) profit_loss_discontinued_operations=YearlyValue(CFY=-150000.0, PFY=-275000.0) total_profit_loss=YearlyValue(CFY=3770000.0, PFY=3070000.0) profit_loss_attributable_to_owners=YearlyValue(CFY=3620000.0, PFY=2950000.0) profit_loss_attributable_to_non_controlling=YearlyValue(CFY=150000.0, PFY=120000.0)


In [24]:
from devtools import debug
debug(result)

C:\Users\almos\AppData\Local\Temp\ipykernel_19356\702232478.py:2 <module>
    result: AgentRunResult(
        data=StatementOfProfitOrLoss(
            revenue=YearlyValue(
                CFY=12500000.0,
                PFY=11250000.0,
            ),
            other_income=YearlyValue(
                CFY=350000.0,
                PFY=275000.0,
            ),
            employee_benefits_expense=None,
            depreciation_expense=None,
            amortisation_expense=None,
            repairs_maintenance_expense=None,
            sales_marketing_expense=None,
            other_expenses=None,
            other_gains_losses=None,
            finance_costs_net=None,
            share_of_profit_loss_associates=None,
            profit_loss_before_taxation=YearlyValue(
                CFY=4780000.0,
                PFY=4080000.0,
            ),
            income_tax_expense_benefit=YearlyValue(
                CFY=-860000.0,
                PFY=-735000.0,
            ),
          

AgentRunResult(data=StatementOfProfitOrLoss(revenue=YearlyValue(CFY=12500000.0, PFY=11250000.0), other_income=YearlyValue(CFY=350000.0, PFY=275000.0), employee_benefits_expense=None, depreciation_expense=None, amortisation_expense=None, repairs_maintenance_expense=None, sales_marketing_expense=None, other_expenses=None, other_gains_losses=None, finance_costs_net=None, share_of_profit_loss_associates=None, profit_loss_before_taxation=YearlyValue(CFY=4780000.0, PFY=4080000.0), income_tax_expense_benefit=YearlyValue(CFY=-860000.0, PFY=-735000.0), profit_loss_discontinued_operations=YearlyValue(CFY=-150000.0, PFY=-275000.0), total_profit_loss=YearlyValue(CFY=3770000.0, PFY=3070000.0), profit_loss_attributable_to_owners=YearlyValue(CFY=3620000.0, PFY=2950000.0), profit_loss_attributable_to_non_controlling=YearlyValue(CFY=150000.0, PFY=120000.0)))